# IPEDS Analysis - Texas State University
**Questions to Answer:**
1. What factors contribute most to student retention rates at Texas State University, and how can interventions be tailored to improve retention?
2. How does the availability of financial aid and scholarships correlate with graduation rates among different demographic groups within the Bobcat Community?

---

## Import Dependencies

In [3]:
import sys
sys.executable


'/Users/robertzygmuntlehr/opt/anaconda3/bin/python'

In [18]:
# Install & Import Dependencies
%pip install openpyxl
%pip show openpyxl
import pandas as pd
import openpyxl
import os
import re
import shutil
import zipfile
import matplotlib.pyplot as plt
# Jupyter Notebook Magic function
%matplotlib inline

Note: you may need to restart the kernel to use updated packages.
Name: openpyxl
Version: 3.1.2
Summary: A Python library to read/write Excel 2010 xlsx/xlsm files
Home-page: https://openpyxl.readthedocs.io
Author: See AUTHORS
Author-email: charlie.clark@clark-consulting.eu
License: MIT
Location: /Users/robertzygmuntlehr/opt/anaconda3/lib/python3.9/site-packages
Requires: et-xmlfile
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## File Handling

In [5]:
# Import Data
df_admission_2022 = pd.read_csv('../../data/csv/adm2022.csv', delimiter = ',')
df_fallEnrollment_2022 = pd.read_csv('../../data/csv/ef2022d.csv', delimiter = ',')


In [6]:
df_admission_2022.head(3)

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,...,XACTEN50,ACTEN50,XACTEN75,ACTEN75,XACTMT25,ACTMT25,XACTMT50,ACTMT50,XACTMT75,ACTMT75
0,100654,1,3,1,3,3,3,5,1,3,...,R,16.0,R,20.0,R,15.0,R,16.0,R,18.0
1,100663,1,3,1,1,3,3,5,3,3,...,R,27.0,R,33.0,R,20.0,R,25.0,R,28.0
2,100706,1,3,1,3,3,3,5,1,3,...,R,28.0,R,34.0,R,23.0,R,26.0,R,30.0


In [7]:
df_fallEnrollment_2022.head(3)

,UNITID,XGRCOHRT,GRCOHRT,XUGENTER,UGENTERN,XPGRCOHR,PGRCOHRT,XRRFTCT,RRFTCT,XRRFTEX,...,XRRPTIN,RRPTIN,XRRPTCTA,RRPTCTA,XRET_NMP,RET_NMP,XRET_PCP,RET_PCP,XSTUFACR,STUFACR
0,100654,R,1547.0,R,1771.0,R,87.0,R,1417.0,R,...,R,0.0,R,60.0,R,15.0,R,25.0,R,19.0
1,100663,R,2172.0,R,3477.0,R,62.0,R,2353.0,R,...,R,0.0,R,54.0,R,29.0,R,54.0,R,18.0
2,100690,A,NaN,A,NaN,A,NaN,A,NaN,A,...,R,0.0,R,0.0,R,0.0,A,NaN,R,12.0


In [9]:
df_summaryStatistics_admission_2022 = pd.read_excel('/Users/robertzygmuntlehr/Desktop/Projects/Texas State University Application/Texas-State-University-IPEDS-Report/data/excel/adm2022.xlsx', sheet_name='Statistics')
df_summaryStatistics_fallEnrollment_2022 = pd.read_excel('/Users/robertzygmuntlehr/Desktop/Projects/Texas State University Application/Texas-State-University-IPEDS-Report/data/csv/ef2022d.xlsx', sheet_name='Statistics')
df_summaryVarlist_admission_2022 = pd.read_excel('/Users/robertzygmuntlehr/Desktop/Projects/Texas State University Application/Texas-State-University-IPEDS-Report/data/excel/adm2022.xlsx', sheet_name='varlist')
df_summaryVarlist_fallEnrollment_2022 = pd.read_excel('/Users/robertzygmuntlehr/Desktop/Projects/Texas State University Application/Texas-State-University-IPEDS-Report/data/csv/ef2022d.xlsx', sheet_name='varlist')

## Universities of Interest:

In [10]:
dictionary_univerisites_id_interest = {
    228459: "Texas State University",
    105330: "Northern Arizona University",
    110510: "California State University-San Bernardino",
    110556: "California State University-Fresno",
    110565: "Califronia State University-Fullerton",
    110583: "Califronia State University-Long Beach",
    122409: "San Diego State University",
    122597: "San Francisco State University",
    133669: "Florida Atlantic University",
    139931: "Georgia Southern University",
    142115: "Boise State University",
    145813: "Illinois State University",
    150136: "Ball State University",
    151111: "Indiana University-Purdue University-Indianapolis",
    184782: "Rowan University",
    185590: "Montclair State University",
    198464: "East Carolina University",
    199139: "University of North Carolina at Charlotte",
    199148: "University of North Carolina at Greensboro",
    199218: "University of North Carolina Wilmington",
    204024: "Miami University-Oxford",
    209807: "Portland State University",
    216764: "West Chester University of Pennsylvania",
    217484: "University of Rhode Island",
    220978: "Middle Tennesse State University",
    225511: "University of Houston",
    227216: "University of North Texas",
    227368: "The University of Texas Rio Grande Valley",
    227881: "Sam Houston State University",
    228723: "Texas A & M University-College Station",
    228769: "The University of Texas at Arlington",
    228778: "The University of Texas at Austin",
    228787: "The University of Texas at Dallas",
    228796: "The University of Texas at El Paso",
    229115: "Texas Tech University",
    232423: "James Madison University",
    483124: "Arizona State University Digital Immersion",
    486840: "Kennesaw State University"
}

In [11]:
# Extract the IDs from the dictionary
university_ids = list(dictionary_univerisites_id_interest.keys())

# Filter the DataFrame
df_filtered_universities = df_admission_2022.loc[df_admission_2022['UNITID'].isin(university_ids)]

df_filtered_universities

,UNITID,ADMCON1,ADMCON2,ADMCON3,ADMCON4,ADMCON5,ADMCON6,ADMCON7,ADMCON8,ADMCON9,...,XACTEN50,ACTEN50,XACTEN75,ACTEN75,XACTMT25,ACTMT25,XACTMT50,ACTMT50,XACTMT75,ACTMT75
33,105330,1,3,1,3,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
67,110510,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
71,110556,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
72,110565,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
74,110583,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
129,122409,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
132,122597,1,3,1,1,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN
216,133669,1,5,1,1,5,3,1,1,3,...,R,22.0,R,25.0,R,17.0,R,20.0,R,24.0
268,139931,1,5,1,1,3,3,1,1,3,...,R,20.0,R,23.0,R,17.0,R,19.0,R,22.0
297,142115,1,3,1,3,3,3,3,1,3,...,A,NaN,A,NaN,A,NaN,A,NaN,A,NaN


In [ ]:
df_retention_rate_filtered_universities = pd.DataFrame
for university_id in df_filtered_universities:
    df_fallEnrollment_2022['RET_PCP']
    df_fallEnrollment_2022['RET_PCF']

In [12]:
df_summaryStatistics_admission_2022

,varnumber,varname,n,sum,mean,min,max,varTitle
0,11016,APPLCN,1991,13172188,6615.865394,0,149801,Applicants total
1,10566,APPLCNM,1991,5616488,2820.938222,0,68595,Applicants men
2,10571,APPLCNW,1991,7516515,3775.246107,0,81206,Applicants women
3,10573,APPLCNAN,722,22104,30.614958,0,1407,Applicants another gender
4,10574,APPLCNUN,1991,17081,8.579106,0,1375,Applicants gender unknown
5,11021,ADMSSN,1964,7701420,3921.293279,0,61739,Admissions total
6,10576,ADMSSNM,1892,3119288,1648.672304,0,30036,Admissions men
7,10581,ADMSSNW,1874,4566864,2436.960512,0,31628,Admissions women
8,10584,ADMSSNAN,377,9870,26.180371,0,422,Admissions another gender
9,10585,ADMSSNUN,1991,5398,2.711200,0,473,Admissions gender unknown


In [13]:
df_summaryStatistics_fallEnrollment_2022

,varnumber,varname,n,sum,mean,min,max,varTitle
0,21001,GRCOHRT,3151,2242594,711.708664,1,14980,Full-time first-time degree/certificate-seekin...
1,21006,UGENTERN,3151,4725821,1499.784513,1,38321,Total entering students at the undergraduate l...
2,21011,PGRCOHRT,3151,165110,52.399238,0,100,Current year GRS cohort as a percent of enteri...
3,21026,RRFTCT,5406,2138057,395.497040,0,14097,Full-time fall 2021 cohort
4,21031,RRFTEX,5406,6441,1.191454,0,2317,Exclusions from full-time fall 2021 cohort
5,21033,RRFTIN,5406,207,0.038291,0,95,Inclusions to the full-time fall 2021 cohort
6,21036,RRFTCTA,5406,2131823,394.343877,0,13931,Full-time adjusted fall 2021 cohort
7,21041,RET_NMF,5406,1631131,301.726045,0,11892,Students from the full-time adjusted fall 2021...
8,21016,RET_PCF,5069,361229,71.262379,0,100,"Full-time retention rate, 2022"
9,21046,RRPTCT,3726,370122,99.334944,0,6795,Part-time fall 2021 cohort


In [14]:
df_summaryVarlist_admission_2022

,varnumber,varname,DataType,Fieldwidth,format,imputationvar,varTitle
0,1,UNITID,N,6,Cont,NaN,Unique identification number of the institution
1,10521,ADMCON1,N,2,Disc,NaN,Secondary school GPA
2,10526,ADMCON2,N,2,Disc,NaN,Secondary school rank
3,10531,ADMCON3,N,2,Disc,NaN,Secondary school record
4,10536,ADMCON4,N,2,Disc,NaN,Completion of college-preparatory program
5,10541,ADMCON5,N,2,Disc,NaN,Recommendations
6,10546,ADMCON6,N,2,Disc,NaN,Formal demonstration of competencies
7,10551,ADMCON7,N,2,Disc,NaN,Admission test scores
8,10556,ADMCON8,N,2,Disc,NaN,English Proficiency Test
9,10559,ADMCON9,N,2,Disc,NaN,"Other Test (Wonderlic, WISC-III, etc.)"


In [15]:
# Unique values in the 'varname' feature of the original DataFrame
original_varnames = set(df_summaryVarlist_admission_2022['varname'])

# Feature names of the filtered DataFrame
filtered_feature_names = set(df_filtered_universities.columns)

# Check if there's a 1:1 matching
if len(original_varnames) == len(filtered_feature_names):
    print("There is a 1:1 matching between the 'varname' feature and the filtered DataFrame's feature names.")
else:
    print("There is not a 1:1 matching between the 'varname' feature and the filtered DataFrame's feature names.")


There is not a 1:1 matching between the 'varname' feature and the filtered DataFrame's feature names.


In [16]:
df_summaryVarlist_fallEnrollment_2022

,varnumber,varname,DataType,Fieldwidth,format,imputationvar,varTitle
0,1,UNITID,N,6,Cont,NaN,Unique identification number of the institution
1,21001,GRCOHRT,N,6,Cont,XGRCOHRT,Full-time first-time degree/certificate-seekin...
2,21006,UGENTERN,N,6,Cont,XUGENTER,Total entering students at the undergraduate l...
3,21011,PGRCOHRT,N,3,Cont,XPGRCOHR,Current year GRS cohort as a percent of enteri...
4,21026,RRFTCT,N,6,Cont,XRRFTCT,Full-time fall 2021 cohort
5,21031,RRFTEX,N,6,Cont,XRRFTEX,Exclusions from full-time fall 2021 cohort
6,21033,RRFTIN,N,6,Cont,XRRFTIN,Inclusions to the full-time fall 2021 cohort
7,21036,RRFTCTA,N,6,Cont,XRRFTCTA,Full-time adjusted fall 2021 cohort
8,21041,RET_NMF,N,6,Cont,XRET_NMF,Students from the full-time adjusted fall 2021...
9,21016,RET_PCF,N,3,Cont,XRET_PCF,"Full-time retention rate, 2022"


---
### Metadata: (2011 - 2022)
- `Full-time retention rate:` (Fall enrollment/retention rates)
Variable Description
The full-time retention rate is the percent of the (fall full-time cohort from the prior year minus exclusions from the fall full-time cohort) that re-enrolled at the institution as either full- or part-time in the current year
Variable Sources
IPEDS, Spring 2023, Fall Enrollment component
No Value sets are available for this variable
- `Part-time retention rate:` (Fall enrollment/retention rates)
Variable Description
The part-time retention rate is the percent of the (fall part-time cohort from the prior year minus exclusions from the fall part-time cohort) that re-enrolled at the institution as either full- or part-time in the current year
Variable Sources
IPEDS, Spring 2023, Fall Enrollment component
No Value sets are available for this variable
- `Reporting method for student charges, graduation rates, retention rates and student financial aid:` (1980-81 to reporting year)
1	Student charges for full academic year and fall GR/SFA/retention rate cohort		
3	Student charges for full academic year and full-year GR/SFA/retention rate cohort		
2	Student charges by program and full-year GR/SFA/retention rate cohort		
-1	Not reported		
-2	Not applicable		
Variable Description
Reporting method identifies how institutions report student charges (full academic year or by program) and the type of cohort (fall or full-year) used to report graduation rate, retention rate and student financial aid data.
- `Response status - total entering class and retention rates:` (1987 to curretn year)
1	Respondent		
2	Partial respondent, imputed		
4	Nonrespondent, imputed		
5	Nonrespondent, not imputed		
-2	Not applicable		
-9	Not active		
Variable Description
Status total entering class
Variable Sources
IPEDS, Spring 2023, Fall Enrollment component
- `Full-time retention rate:` (Fall 2003 to current year) 
Variable Description
The full-time retention rate is the percent of the (fall full-time cohort from the prior year minus exclusions from the fall full-time cohort) that re-enrolled at the institution as either full- or part-time in the current year
Variable Sources
IPEDS, Spring 2023, Fall Enrollment component
No Value sets are available for this variable
- `Part-time retention rate:`(Fall 2003 to current year)
Variable Description
The part-time retention rate is the percent of the (fall part-time cohort from the prior year minus exclusions from the fall part-time cohort) that re-enrolled at the institution as either full- or part-time in the current year
Variable Sources
IPEDS, Spring 2023, Fall Enrollment component
No Value sets are available for this variable

---

### Comparing to similar universities sharing these parameters:
**Criteria Summary:**
- Miscellaneous Indicators: All institutions
- Sector: Public, 4-year or above
- State or other jurisdiction: Texas
- Degree-granting status: Degree-granting
- Highest degree offered: Doctor's degree - research/scholarship and professional practice
- Institutional category: Degree-granting, primarily baccalaureate or above
- Degree of urbanization (Locale): City: Small; City: Midsize; City: Large
- Institution size category: 20,000 and above
- Reporting method (academic/hybrid/program): Student charges for full academic year and fall GR/SFA/retention rate cohort
- Has full-time first-time undergraduates: Yes
- All programs offered completely via distance education: No
- Adding select peer institutions as determined by the NCES:
    - 225511: University of Houston	TX
    - 227216: University of North Texas	TX
    - 227368: The University of Texas Rio Grande Valley	TX
    - 228723: Texas A & M University-College Station	TX
    - 228769: The University of Texas at Arlington	TX
    - 228778: The University of Texas at Austin	TX
    - 228787: The University of Texas at Dallas	TX
    - 228796: The University of Texas at El Paso	TX
    - 229115: Texas Tech University	TX
    - 105330: Northern Arizona University	AZ
    - 110510: California State University-San Bernardino	CA
    - 110556: California State University-Fresno	CA
    - 110565: California State University-Fullerton	CA
    - 110583: California State University-Long Beach	CA
    - 122409: San Diego State University	CA
    - 122597: San Francisco State University	CA
    - 133669: Florida Atlantic University	FL
    - 139931: Georgia Southern University	GA
    - 142115: Boise State University	ID
    - 145813: Illinois State University	IL
